In [1]:
import os.path as osp

import torch
import torch.nn.functional as F
from itertools import groupby
from sklearn.preprocessing import OneHotEncoder
import numpy as np 
import sys
sys.path.insert(0, '/Users/mridulag/Downloads/pytorch_geometric-master/')

from torch_geometric.datasets import TUDataset
import torch_geometric.transforms as T
from torch_geometric.nn.conv import GCNConv, ChebConv
from torch_scatter import scatter_add

from IPython.core.debugger import Tracer; debug_here = Tracer()

# from torch_geometric.utils.loop import *

/Users/mridulag/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:16: DeprecationWarning: `Tracer` is deprecated since version 5.1, directly use `IPython.core.debugger.Pdb.set_trace()`
  app.launch_new_instance()


In [2]:
path = '/Users/mridulag/Downloads/pytorch_geometric-master/examples/../data/MUTAG';
dataset = TUDataset(path, name='MUTAG', use_node_attr=False)#.shuffle()
print(len(dataset))
data = dataset[0]
print(dataset)

init
if
188
MUTAG(188)


In [3]:
#Creating nodes labels using node features 
t1 = torch.ones(data.num_nodes,dtype=torch.int64)
for i in range(data.num_features):
    t1 = t1 + (torch.tensor(data.x,dtype=torch.int64)[:,i]*i)
print(t1)

tensor([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 3, 3])


/Users/mridulag/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:4: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  after removing the cwd from sys.path.


In [4]:
#updating data.y to have node labels
data.y = t1

In [5]:
#updating data.x (node features) with node degrees
degree = [len(list(group)) for key, group in groupby(data.edge_index[0])]
print(degree)
integer_encoded = np.array(degree)
print(integer_encoded)

onehot_encoder = OneHotEncoder(sparse=False)
integer_encoded = integer_encoded.reshape(len(integer_encoded), 1)
onehot_encoded = onehot_encoder.fit_transform(integer_encoded)
print(onehot_encoded)
data.x =  torch.tensor(onehot_encoded,dtype = torch.float32)

[2, 2, 2, 3, 3, 2, 2, 2, 3, 3, 2, 2, 3, 2, 3, 1, 1]
[2 2 2 3 3 2 2 2 3 3 2 2 3 2 3 1 1]
[[0. 1. 0.]
 [0. 1. 0.]
 [0. 1. 0.]
 [0. 0. 1.]
 [0. 0. 1.]
 [0. 1. 0.]
 [0. 1. 0.]
 [0. 1. 0.]
 [0. 0. 1.]
 [0. 0. 1.]
 [0. 1. 0.]
 [0. 1. 0.]
 [0. 0. 1.]
 [0. 1. 0.]
 [0. 0. 1.]
 [1. 0. 0.]
 [1. 0. 0.]]


/Users/mridulag/anaconda3/lib/python3.6/site-packages/sklearn/preprocessing/_encoders.py:368: FutureWarning: The handling of integer data will change in version 0.22. Currently, the categories are determined based on the range [0, max(values)], while in the future they will be determined based on the unique values.
If you want the future behaviour and silence this warning, you can specify "categories='auto'".
In case you used a LabelEncoder before this OneHotEncoder to convert the categories to integers, then you can now use the OneHotEncoder directly.
  warnings.warn(msg, FutureWarning)


In [6]:
print(type(data.x))
print(data.y)
print(data.edge_index.shape)

<class 'torch.Tensor'>
tensor([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 3, 3])
torch.Size([2, 38])


In [7]:
def sample_mask(index, num_nodes):
    mask = torch.zeros((num_nodes, ), dtype=torch.uint8)
    mask[index] = 1
    return mask

In [8]:
train_index = torch.arange(5, dtype=torch.long)
val_index = torch.arange(5, 12, dtype=torch.long)
test_index = torch.arange(12, 17, dtype=torch.long)


print(train_index)
# print(val_index)

train_mask = sample_mask(train_index, num_nodes=data.y.size(0))
val_mask = sample_mask(val_index, num_nodes=data.y.size(0))
test_mask = sample_mask(test_index, num_nodes=data.y.size(0))

print(train_mask,val_mask,test_mask)

data.train_mask = train_mask
data.val_mask = val_mask
data.test_mask = test_mask

tensor([0, 1, 2, 3, 4])
tensor([1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], dtype=torch.uint8) tensor([0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0], dtype=torch.uint8) tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1], dtype=torch.uint8)


In [9]:
clusters={}

In [10]:
epocnum = 200

class Net(torch.nn.Module):
#     i = 0
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = GCNConv(3, 7, cached=True)
        self.conv2 = GCNConv(7, 3, cached=True)
        # self.conv1 = ChebConv(data.num_features, 16, K=2)
        # self.conv2 = ChebConv(16, data.num_features, K=2)
        self.i = 0
        self.fw1 = torch.zeros([17,7],dtype=None)

    def forward(self):
        
        self.i = self.i + 1
        x, edge_index = data.x, data.edge_index
        x_f = x    #Features
#         debug_here()
        x = self.conv1(x, edge_index)
        x_c1 = x
        x = F.relu(x)

        x_h1 = x   #H1
        x = F.dropout(x, training=self.training)
        x = self.conv2(x, edge_index)
        x_h2 = x   #H2
        x = F.log_softmax(x, dim=1)
        if self.i == epocnum*2-2:
            self.fw1 = torch.matmul(x_f, model.conv1.weight)
            print(self.fw1.shape)
                
            clusters["nodes"] = []
            i = 0
            numnodes = x_f.shape[0]
            
            while i < numnodes:
                node = {"id": str(i), "group": 0}
                j = 0
                while j < 3:
                    fi = "f" + str(j+1)
                    node[fi]=str(float(x_f[i][j]))
                    fi = "h2" + str(j+1)
                    node[fi]=str(round(float(x_h2[i][j]),2))
                    j +=1
    
                j = 0
                while j < 7:
                    fi = "h1" + str(j+1)
                    node[fi]=str(round(float(x_h1[i][j]),2))
                    j +=1             

                j = 0
                while j < 7:
                    fi = "fw1"+str(j+1)
                    node[fi]=str(round(float(self.fw1[i][j]),2))
                    j +=1             
#                 i += 1
                #print(node)
#                 clusters["nodes"].append(node)
                
                j = 0
                while j < 7:
                    fi = "br"+str(j+1)
                    node[fi]=str(round(float(x_c1[i][j]),2))
                    j +=1             
                i += 1
                print(node)
                clusters["nodes"].append(node)
        
        
            print("Features", x_f)
            print("Weights", torch.round(model.conv1.weight.data*1000)/1000)
            print("Before Relu", torch.round(x_c1*100)/100)
            print("After Relu", x_h1)

        return x

In [11]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model, data = Net().to(device), data.to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=0.01, weight_decay=5e-4)

In [12]:
def train():
    model.train()
    optimizer.zero_grad()
#    print("model", model())
    #print("Actual", model()[data.train_mask], "Pred", data.y[data.train_mask])
    #print("Actual", model(), "Pred", data.y[data.train_mask].shape)
    F.nll_loss(model()[data.train_mask], data.y[data.train_mask]).backward()
#     output =  model()
#     F.nll_loss(output, data.y).backward()
    optimizer.step()


def test():
    model.eval()
    logits, accs = model(), []
    for _, mask in data('train_mask', 'val_mask', 'test_mask'):
        pred = logits[mask].max(1)[1]
        acc = pred.eq(data.y[mask]).sum().item() / mask.sum().item()
        accs.append(acc)
    return accs

In [13]:
best_val_acc = test_acc = 0

for epoch in range(1, epocnum):
    train()
    #break
    train_acc, val_acc, tmp_test_acc = test()
    if val_acc > best_val_acc:
        best_val_acc = val_acc
        test_acc = tmp_test_acc
    log = 'Epoch: {:03d}, Train: {:.4f}, Val: {:.4f}, Test: {:.4f}'
#     print(log.format(epoch, train_acc, best_val_acc, test_acc))

forward final tensor([[-0.1600, -0.5400, -0.1300,  0.5600,  0.6300,  0.0100,  0.3700],
        [-0.1600, -0.5400, -0.1300,  0.5600,  0.6300,  0.0100,  0.3700],
        [-0.2000, -0.5700, -0.0000,  0.1500,  0.2000, -0.1500,  0.0600],
        [-0.3000, -0.7100,  0.1800, -0.4100, -0.4000, -0.4100, -0.3800],
        [-0.2600, -0.6800,  0.0700, -0.0600, -0.0200, -0.2700, -0.1200],
        [-0.2000, -0.5700, -0.0000,  0.1500,  0.2000, -0.1500,  0.0600],
        [-0.2000, -0.5700, -0.0000,  0.1500,  0.2000, -0.1500,  0.0600],
        [-0.2000, -0.5700, -0.0000,  0.1500,  0.2000, -0.1500,  0.0600],
        [-0.2600, -0.6800,  0.0700, -0.0600, -0.0200, -0.2700, -0.1200],
        [-0.3000, -0.7100,  0.1800, -0.4100, -0.4000, -0.4100, -0.3800],
        [-0.2000, -0.5700, -0.0000,  0.1500,  0.2000, -0.1500,  0.0600],
        [-0.2000, -0.5700, -0.0000,  0.1500,  0.2000, -0.1500,  0.0600],
        [-0.2600, -0.6800,  0.0700, -0.0600, -0.0200, -0.2700, -0.1200],
        [-0.2500, -0.6000,  0.1300, -

forward final tensor([[ 0.0700,  0.1700, -0.0300],
        [ 0.0700,  0.1700, -0.0300],
        [-0.0300,  0.1300, -0.0900],
        [-0.1400,  0.0900, -0.1700],
        [-0.1200,  0.0900, -0.1300],
        [-0.0200,  0.1200, -0.0700],
        [-0.1000,  0.0800, -0.0900],
        [-0.1000,  0.0800, -0.0900],
        [-0.1300,  0.0900, -0.1600],
        [-0.1400,  0.0900, -0.1700],
        [-0.1100,  0.0800, -0.1100],
        [-0.1000,  0.0800, -0.0900],
        [-0.1200,  0.1200, -0.2500],
        [-0.1200,  0.0800, -0.1600],
        [-0.1200,  0.2000, -0.4700],
        [-0.1000,  0.1800, -0.4200],
        [-0.1000,  0.1800, -0.4200]], grad_fn=<DivBackward0>)
forward final tensor([[-0.1000, -0.4800, -0.0300,  0.4800,  0.5200,  0.1200,  0.3000],
        [-0.1000, -0.4800, -0.0300,  0.4800,  0.5200,  0.1200,  0.3000],
        [-0.1500, -0.5200,  0.0900,  0.0700,  0.0900, -0.0500, -0.0000],
        [-0.2300, -0.6500,  0.2700, -0.5000, -0.5100, -0.3000, -0.4300],
        [-0.2000, -0.6200,

forward final tensor([[-0.2800,  0.1300,  0.0800],
        [-0.2800,  0.1300,  0.0800],
        [-0.2400,  0.0900, -0.0900],
        [-0.1400,  0.1000, -0.1700],
        [-0.0900,  0.1000, -0.1100],
        [-0.1200,  0.1200,  0.0900],
        [-0.1400,  0.1200, -0.1500],
        [-0.1800,  0.1400, -0.2200],
        [-0.2800,  0.1500, -0.3500],
        [-0.2200,  0.1300, -0.2600],
        [-0.2400,  0.1400, -0.2500],
        [-0.2100,  0.1300, -0.2100],
        [-0.1700,  0.1300, -0.2500],
        [-0.2400,  0.1400, -0.3300],
        [-0.2100,  0.2500, -0.6500],
        [-0.0600,  0.2300, -0.4800],
        [-0.1500,  0.1500, -0.3900]], grad_fn=<DivBackward0>)
forward final tensor([[-0.0800, -0.4500,  0.0200,  0.4700,  0.4700,  0.1600,  0.3100],
        [-0.0800, -0.4500,  0.0200,  0.4700,  0.4700,  0.1600,  0.3100],
        [-0.1200, -0.4900,  0.1400,  0.0700,  0.0400, -0.0000,  0.0100],
        [-0.2000, -0.6100,  0.3300, -0.5100, -0.5600, -0.2500, -0.4200],
        [-0.1600, -0.5800,

forward final tensor([[-0.0500, -0.4300,  0.0800,  0.4900,  0.4200,  0.2100,  0.3100],
        [-0.0500, -0.4300,  0.0800,  0.4900,  0.4200,  0.2100,  0.3100],
        [-0.0900, -0.4600,  0.2000,  0.0900, -0.0000,  0.0500,  0.0200],
        [-0.1700, -0.5800,  0.3900, -0.4800, -0.6000, -0.2000, -0.4000],
        [-0.1400, -0.5500,  0.2800, -0.1300, -0.2300, -0.0600, -0.1500],
        [-0.0900, -0.4600,  0.2000,  0.0900, -0.0000,  0.0500,  0.0200],
        [-0.0900, -0.4600,  0.2000,  0.0900, -0.0000,  0.0500,  0.0200],
        [-0.0900, -0.4600,  0.2000,  0.0900, -0.0000,  0.0500,  0.0200],
        [-0.1400, -0.5500,  0.2800, -0.1300, -0.2300, -0.0600, -0.1500],
        [-0.1700, -0.5800,  0.3900, -0.4800, -0.6000, -0.2000, -0.4000],
        [-0.0900, -0.4600,  0.2000,  0.0900, -0.0000,  0.0500,  0.0200],
        [-0.0900, -0.4600,  0.2000,  0.0900, -0.0000,  0.0500,  0.0200],
        [-0.1400, -0.5500,  0.2800, -0.1300, -0.2300, -0.0600, -0.1500],
        [-0.1400, -0.5000,  0.3200, -

forward final tensor([[-0.5600,  0.2900, -0.4800],
        [-0.4500,  0.2200, -0.3300],
        [-0.3000,  0.2300, -0.2500],
        [-0.3200,  0.2300, -0.4300],
        [-0.3800,  0.2800, -0.5100],
        [-0.4900,  0.2600, -0.5300],
        [-0.4100,  0.3000, -0.5700],
        [-0.3200,  0.2600, -0.4300],
        [-0.3300,  0.2600, -0.4500],
        [-0.2200,  0.2300, -0.3400],
        [-0.2200,  0.2500, -0.3900],
        [-0.1900,  0.2300, -0.3500],
        [-0.3400,  0.3300, -0.6100],
        [-0.2400,  0.1900, -0.2800],
        [-0.6200,  0.5700, -1.2400],
        [-0.4800,  0.5800, -1.1300],
        [-0.5300,  0.4600, -1.0000]], grad_fn=<DivBackward0>)
forward final tensor([[-0.0200, -0.3900,  0.1600,  0.5500,  0.3700,  0.2800,  0.3600],
        [-0.0200, -0.3900,  0.1600,  0.5500,  0.3700,  0.2800,  0.3600],
        [-0.0600, -0.4200,  0.2700,  0.1400, -0.0500,  0.1100,  0.0600],
        [-0.1400, -0.5400,  0.4600, -0.4200, -0.6500, -0.1400, -0.3500],
        [-0.1000, -0.5100,

...

forward final tensor([[-0.0600,  0.8300,  0.9400,  1.6500,  0.4000,  1.0900,  1.5800],
        [-0.0600,  0.8300,  0.9400,  1.6500,  0.4000,  1.0900,  1.5800],
        [-0.0500,  0.7900,  1.0300,  1.2700, -0.0100,  0.9400,  1.3200],
        [-0.0400,  0.7900,  1.2200,  0.8300, -0.5900,  0.7800,  1.0400],
        [-0.0500,  0.8200,  1.1500,  1.1600, -0.2300,  0.9100,  1.2700],
        [-0.0500,  0.7900,  1.0300,  1.2700, -0.0100,  0.9400,  1.3200],
        [-0.0500,  0.7900,  1.0300,  1.2700, -0.0100,  0.9400,  1.3200],
        [-0.0500,  0.7900,  1.0300,  1.2700, -0.0100,  0.9400,  1.3200],
        [-0.0500,  0.8200,  1.1500,  1.1600, -0.2300,  0.9100,  1.2700],
        [-0.0400,  0.7900,  1.2200,  0.8300, -0.5900,  0.7800,  1.0400],
        [-0.0500,  0.7900,  1.0300,  1.2700, -0.0100,  0.9400,  1.3200],
        [-0.0500,  0.7900,  1.0300,  1.2700, -0.0100,  0.9400,  1.3200],
        [-0.0500,  0.8200,  1.1500,  1.1600, -0.2300,  0.9100,  1.2700],
        [-0.0400,  0.7600,  1.1100,  

forward final tensor([[-4.5000,  4.7700, -5.8700],
        [-5.4700,  4.2600, -5.3000],
        [-3.7900,  4.2400, -5.0400],
        [-3.8600,  4.0900, -4.6700],
        [-3.6300,  5.1800, -5.7900],
        [-3.7400,  3.5700, -4.1200],
        [-3.4900,  3.5400, -4.1200],
        [-4.9800,  4.8600, -5.6700],
        [-4.6700,  4.9600, -5.7500],
        [-3.6200,  4.0800, -4.4400],
        [-3.8500,  3.8500, -4.3800],
        [-3.4900,  4.0700, -4.6000],
        [-4.4000,  5.4200, -6.2600],
        [-4.2600,  5.3900, -6.0300],
        [-3.4400,  3.6600, -4.3400],
        [-3.1600,  2.4800, -3.0100],
        [-1.7000,  1.7400, -1.9900]], grad_fn=<DivBackward0>)
forward final tensor([[-0.0600,  0.9000,  0.9500,  1.6500,  0.4000,  1.0900,  1.5900],
        [-0.0600,  0.9000,  0.9500,  1.6500,  0.4000,  1.0900,  1.5900],
        [-0.0500,  0.8600,  1.0300,  1.2700, -0.0100,  0.9400,  1.3200],
        [-0.0400,  0.8600,  1.2200,  0.8300, -0.5900,  0.7800,  1.0400],
        [-0.0500,  0.8900,

forward final tensor([[-0.0600,  0.9500,  0.9500,  1.6500,  0.4000,  1.1000,  1.5900],
        [-0.0600,  0.9500,  0.9500,  1.6500,  0.4000,  1.1000,  1.5900],
        [-0.0500,  0.9100,  1.0300,  1.2700, -0.0100,  0.9400,  1.3300],
        [-0.0400,  0.9000,  1.2200,  0.8400, -0.5900,  0.7800,  1.0400],
        [-0.0500,  0.9400,  1.1500,  1.1600, -0.2300,  0.9100,  1.2800],
        [-0.0500,  0.9100,  1.0300,  1.2700, -0.0100,  0.9400,  1.3300],
        [-0.0500,  0.9100,  1.0300,  1.2700, -0.0100,  0.9400,  1.3300],
        [-0.0500,  0.9100,  1.0300,  1.2700, -0.0100,  0.9400,  1.3300],
        [-0.0500,  0.9400,  1.1500,  1.1600, -0.2300,  0.9100,  1.2800],
        [-0.0400,  0.9000,  1.2200,  0.8400, -0.5900,  0.7800,  1.0400],
        [-0.0500,  0.9100,  1.0300,  1.2700, -0.0100,  0.9400,  1.3300],
        [-0.0500,  0.9100,  1.0300,  1.2700, -0.0100,  0.9400,  1.3300],
        [-0.0500,  0.9400,  1.1500,  1.1600, -0.2300,  0.9100,  1.2800],
        [-0.0400,  0.8600,  1.1100,  

forward final tensor([[-0.0600,  1.0300,  0.9500,  1.6500,  0.4000,  1.1000,  1.6000],
        [-0.0600,  1.0300,  0.9500,  1.6500,  0.4000,  1.1000,  1.6000],
        [-0.0500,  0.9800,  1.0300,  1.2800, -0.0100,  0.9500,  1.3300],
        [-0.0400,  0.9700,  1.2200,  0.8400, -0.5800,  0.7800,  1.0500],
        [-0.0500,  1.0200,  1.1500,  1.1700, -0.2300,  0.9200,  1.2800],
        [-0.0500,  0.9800,  1.0300,  1.2800, -0.0100,  0.9500,  1.3300],
        [-0.0500,  0.9800,  1.0300,  1.2800, -0.0100,  0.9500,  1.3300],
        [-0.0500,  0.9800,  1.0300,  1.2800, -0.0100,  0.9500,  1.3300],
        [-0.0500,  1.0200,  1.1500,  1.1700, -0.2300,  0.9200,  1.2800],
        [-0.0400,  0.9700,  1.2200,  0.8400, -0.5800,  0.7800,  1.0500],
        [-0.0500,  0.9800,  1.0300,  1.2800, -0.0100,  0.9500,  1.3300],
        [-0.0500,  0.9800,  1.0300,  1.2800, -0.0100,  0.9500,  1.3300],
        [-0.0500,  1.0200,  1.1500,  1.1700, -0.2300,  0.9200,  1.2800],
        [-0.0400,  0.9300,  1.1100,  

forward final tensor([[-4.2800,  4.9400, -5.1500],
        [-4.2800,  4.9400, -5.1500],
        [-3.9100,  4.4200, -4.7100],
        [-4.0300,  4.4500, -4.8700],
        [-4.2600,  4.7000, -5.1300],
        [-4.0200,  4.5400, -4.8200],
        [-3.9100,  4.3100, -4.6800],
        [-3.9100,  4.3100, -4.6800],
        [-4.1300,  4.5700, -5.0000],
        [-4.0300,  4.4500, -4.8700],
        [-3.8100,  4.2000, -4.5700],
        [-3.9100,  4.3100, -4.6800],
        [-3.9100,  4.3400, -4.7300],
        [-3.6200,  3.9800, -4.3400],
        [-3.5100,  3.9500, -4.3100],
        [-2.4100,  2.7000, -2.9200],
        [-2.4100,  2.7000, -2.9200]], grad_fn=<DivBackward0>)
forward final tensor([[-0.0500,  1.2500,  0.9500,  1.6600,  0.4000,  1.1100,  1.6200],
        [-0.0500,  1.2500,  0.9500,  1.6600,  0.4000,  1.1100,  1.6200],
        [-0.0400,  1.1900,  1.0300,  1.2900,  0.0000,  0.9500,  1.3500],
        [-0.0300,  1.1900,  1.2300,  0.8500, -0.5600,  0.7900,  1.0800],
        [-0.0400,  1.2400,

In [14]:
print(clusters["nodes"])

[{'id': '0', 'group': 0, 'f1': '0.0', 'h21': '-4.3', 'f2': '1.0', 'h22': '4.96', 'f3': '0.0', 'h23': '-5.16', 'h11': '0.0', 'h12': '1.26', 'h13': '0.95', 'h14': '1.66', 'h15': '0.4', 'h16': '1.11', 'h17': '1.62', 'fw11': '-0.02', 'fw12': '0.56', 'fw13': '0.42', 'fw14': '1.08', 'fw15': '0.51', 'fw16': '0.54', 'fw17': '0.95', 'br1': '-0.05', 'br2': '1.26', 'br3': '0.95', 'br4': '1.66', 'br5': '0.4', 'br6': '1.11', 'br7': '1.62'}, {'id': '1', 'group': 0, 'f1': '0.0', 'h21': '-4.3', 'f2': '1.0', 'h22': '4.96', 'f3': '0.0', 'h23': '-5.16', 'h11': '0.0', 'h12': '1.26', 'h13': '0.95', 'h14': '1.66', 'h15': '0.4', 'h16': '1.11', 'h17': '1.62', 'fw11': '-0.02', 'fw12': '0.56', 'fw13': '0.42', 'fw14': '1.08', 'fw15': '0.51', 'fw16': '0.54', 'fw17': '0.95', 'br1': '-0.05', 'br2': '1.26', 'br3': '0.95', 'br4': '1.66', 'br5': '0.4', 'br6': '1.11', 'br7': '1.62'}, {'id': '2', 'group': 0, 'f1': '0.0', 'h21': '-3.93', 'f2': '1.0', 'h22': '4.44', 'f3': '0.0', 'h23': '-4.72', 'h11': '0.0', 'h12': '1.21'

In [15]:
edge_index = model.conv1.cached_result[0][0]
print(edge_index)
# norm.view(-1, 1)
# model.fw1.shape

tensor([ 0,  0,  1,  1,  2,  2,  3,  3,  3,  4,  4,  4,  5,  5,  6,  6,  7,  7,
         8,  8,  8,  9,  9,  9, 10, 10, 11, 11, 12, 12, 12, 13, 13, 14, 14, 14,
        15, 16,  0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15,
        16])


In [16]:
print(data.edge_index.shape)
clusters["links"] = []

len1 = len(data.edge_index[0])
for i in range(len1):
    #print(i)
    s = str(int(data.edge_index[0][i]))
    d = str(int(data.edge_index[1][i]))
    node = {"source": s, "target": d, "strength": float(0.95)}
    print(node)
    clusters["links"].append(node)
    
# print(clusters)

torch.Size([2, 38])
{'source': '0', 'target': '1', 'strength': 0.95}
{'source': '0', 'target': '5', 'strength': 0.95}
{'source': '1', 'target': '0', 'strength': 0.95}
{'source': '1', 'target': '2', 'strength': 0.95}
{'source': '2', 'target': '1', 'strength': 0.95}
{'source': '2', 'target': '3', 'strength': 0.95}
{'source': '3', 'target': '2', 'strength': 0.95}
{'source': '3', 'target': '4', 'strength': 0.95}
{'source': '3', 'target': '9', 'strength': 0.95}
{'source': '4', 'target': '3', 'strength': 0.95}
{'source': '4', 'target': '5', 'strength': 0.95}
{'source': '4', 'target': '6', 'strength': 0.95}
{'source': '5', 'target': '0', 'strength': 0.95}
{'source': '5', 'target': '4', 'strength': 0.95}
{'source': '6', 'target': '4', 'strength': 0.95}
{'source': '6', 'target': '7', 'strength': 0.95}
{'source': '7', 'target': '6', 'strength': 0.95}
{'source': '7', 'target': '8', 'strength': 0.95}
{'source': '8', 'target': '7', 'strength': 0.95}
{'source': '8', 'target': '9', 'strength': 0.95}


In [17]:
print(model)
print(model.conv1.weight.data)
print(model.conv2.weight.data)

Net(
  (conv1): GCNConv(3, 7)
  (conv2): GCNConv(7, 3)
)
tensor([[-2.0231e-04,  1.5095e-05, -2.3128e-04,  1.8551e-06,  3.5687e-06,
          4.2888e-05, -1.0617e-04],
        [-2.4540e-02,  5.5582e-01,  4.2116e-01,  1.0791e+00,  5.1418e-01,
          5.3520e-01,  9.4664e-01],
        [ 9.1530e-06,  4.5832e-01,  7.6488e-01, -4.9825e-02, -7.9914e-01,
          8.9463e-02,  1.7882e-01]])
tensor([[ 4.2573e-06,  4.8633e-06,  1.2129e-06],
        [-6.6938e-01,  6.3524e-01, -4.2521e-01],
        [-1.0120e+00,  7.1684e-01, -1.1109e+00],
        [-1.1837e+00,  3.1463e-01, -2.5091e-01],
        [ 2.8225e-01,  5.2420e-01,  1.1697e-01],
        [-6.2513e-02,  1.2405e+00, -1.2165e+00],
        [-1.0126e-01,  6.8916e-01, -9.6299e-01]])


In [18]:
clusters["weights"] = {}
        
clusters["weights"]["conv1"] = []

for data in model.conv1.weight:
    Weights = data.tolist()
    Weights = [round(x,2) for x in Weights]
#     print(Weights)
    clusters["weights"]["conv1"].append(Weights)

clusters["weights"]["conv2"] = []
for data in model.conv2.weight:
    Weights = data.tolist()
    Weights = [round(x,2) for x in Weights]
#     print(Weights)
    clusters["weights"]["conv2"].append(Weights)
    
clusters["weights"]["norm"] = []
norm = model.conv1.cached_result[1]
data = norm.view(-1, 1)
print(data)
Weights = data.tolist()
for data in Weights:
#     Weights = [round(x,2) for x in Weights]
    print(Weights)
    clusters["weights"]["norm"].append(data)

print(clusters["weights"]["norm"])

#print(clusters)

tensor([[0.3333],
        [0.3333],
        [0.3333],
        [0.3333],
        [0.3333],
        [0.2887],
        [0.2887],
        [0.2500],
        [0.2500],
        [0.2500],
        [0.2887],
        [0.2887],
        [0.3333],
        [0.2887],
        [0.2887],
        [0.3333],
        [0.3333],
        [0.2887],
        [0.2887],
        [0.2500],
        [0.2887],
        [0.2500],
        [0.2500],
        [0.2887],
        [0.2887],
        [0.3333],
        [0.3333],
        [0.2887],
        [0.2887],
        [0.2887],
        [0.2500],
        [0.2887],
        [0.2887],
        [0.2500],
        [0.3536],
        [0.3536],
        [0.3536],
        [0.3536],
        [0.3333],
        [0.3333],
        [0.3333],
        [0.2500],
        [0.2500],
        [0.3333],
        [0.3333],
        [0.3333],
        [0.2500],
        [0.2500],
        [0.3333],
        [0.3333],
        [0.2500],
        [0.3333],
        [0.2500],
        [0.5000],
        [0.5000]])
[[0.33333

[[0.3333333134651184], [0.3333333134651184], [0.3333333134651184], [0.3333333134651184], [0.3333333134651184], [0.28867512941360474], [0.28867512941360474], [0.25], [0.25], [0.25], [0.28867512941360474], [0.28867512941360474], [0.3333333134651184], [0.28867512941360474], [0.28867512941360474], [0.3333333134651184], [0.3333333134651184], [0.28867512941360474], [0.28867512941360474], [0.25], [0.28867512941360474], [0.25], [0.25], [0.28867512941360474], [0.28867512941360474], [0.3333333134651184], [0.3333333134651184], [0.28867512941360474], [0.28867512941360474], [0.28867512941360474], [0.25], [0.28867512941360474], [0.28867512941360474], [0.25], [0.3535533845424652], [0.3535533845424652], [0.3535533845424652], [0.3535533845424652], [0.3333333134651184], [0.3333333134651184], [0.3333333134651184], [0.25], [0.25], [0.3333333134651184], [0.3333333134651184], [0.3333333134651184], [0.25], [0.25], [0.3333333134651184], [0.3333333134651184], [0.25], [0.3333333134651184], [0.25], [0.4999999701

In [19]:
tmp = model.fw1
# print(tmp)

EM_List_Nodes = torch.zeros(17,8)
i = 0
for row in tmp.data:
    EM_List_Nodes[i][0] = i
    j = 1
    for item in row:
        EM_List_Nodes[i][j] = item
        j = j+1
    i = i + 1
# print(EM_List_Nodes)

edge_index = model.conv1.cached_result[0]

EM = torch.index_select(EM_List_Nodes, 0, edge_index[0])
# print(EM)

EM_round = torch.round(EM.data*100)/100
# print(EM_round)

EM_List = EM_round.tolist()
# print(EM_List)
 
clusters["weights"]["EM_header"] = ["Node","h1","h2","h3","h4","h5","h6","h7"]
clusters["weights"]["EM"] = EM_List
print(clusters["weights"]["EM"])


[[0.0, -0.019999999552965164, 0.5600000023841858, 0.41999998688697815, 1.0800000429153442, 0.5099999904632568, 0.5400000214576721, 0.949999988079071], [0.0, -0.019999999552965164, 0.5600000023841858, 0.41999998688697815, 1.0800000429153442, 0.5099999904632568, 0.5400000214576721, 0.949999988079071], [1.0, -0.019999999552965164, 0.5600000023841858, 0.41999998688697815, 1.0800000429153442, 0.5099999904632568, 0.5400000214576721, 0.949999988079071], [1.0, -0.019999999552965164, 0.5600000023841858, 0.41999998688697815, 1.0800000429153442, 0.5099999904632568, 0.5400000214576721, 0.949999988079071], [2.0, -0.019999999552965164, 0.5600000023841858, 0.41999998688697815, 1.0800000429153442, 0.5099999904632568, 0.5400000214576721, 0.949999988079071], [2.0, -0.019999999552965164, 0.5600000023841858, 0.41999998688697815, 1.0800000429153442, 0.5099999904632568, 0.5400000214576721, 0.949999988079071], [3.0, 0.0, 0.46000000834465027, 0.7599999904632568, -0.05000000074505806, -0.800000011920929, 0.090

In [20]:
norm = model.conv1.cached_result[1]
NM = norm.view(-1, 1) * EM
NM_round = torch.round(NM*1000)/1000
print(NM_round)
NM_List = NM_round.tolist()

i = 0
for row in NM_List:
    row[0] = EM_List[i][0]
    i = i + 1
    
clusters["weights"]["NM"] = NM_List
clusters["weights"]["NM_header"] = ["Node","h1","h2","h3","h4","h5","h6","h7"]

# print(clusters["weights"]["NM"] )
print(len(NM_List),"*",len(NM_List[0]))

tensor([[ 0.0000e+00, -8.0000e-03,  1.8500e-01,  1.4000e-01,  3.6000e-01,
          1.7100e-01,  1.7800e-01,  3.1600e-01],
        [ 0.0000e+00, -8.0000e-03,  1.8500e-01,  1.4000e-01,  3.6000e-01,
          1.7100e-01,  1.7800e-01,  3.1600e-01],
        [ 3.3300e-01, -8.0000e-03,  1.8500e-01,  1.4000e-01,  3.6000e-01,
          1.7100e-01,  1.7800e-01,  3.1600e-01],
        [ 3.3300e-01, -8.0000e-03,  1.8500e-01,  1.4000e-01,  3.6000e-01,
          1.7100e-01,  1.7800e-01,  3.1600e-01],
        [ 6.6700e-01, -8.0000e-03,  1.8500e-01,  1.4000e-01,  3.6000e-01,
          1.7100e-01,  1.7800e-01,  3.1600e-01],
        [ 5.7700e-01, -7.0000e-03,  1.6000e-01,  1.2200e-01,  3.1200e-01,
          1.4800e-01,  1.5500e-01,  2.7300e-01],
        [ 8.6600e-01,  0.0000e+00,  1.3200e-01,  2.2100e-01, -1.4000e-02,
         -2.3100e-01,  2.6000e-02,  5.2000e-02],
        [ 7.5000e-01,  0.0000e+00,  1.1500e-01,  1.9100e-01, -1.2000e-02,
         -2.0000e-01,  2.2000e-02,  4.5000e-02],
        [ 7.5000

In [21]:
print(clusters["weights"])

{'conv1': [[-0.0, 0.0, -0.0, 0.0, 0.0, 0.0, -0.0], [-0.02, 0.56, 0.42, 1.08, 0.51, 0.54, 0.95], [0.0, 0.46, 0.76, -0.05, -0.8, 0.09, 0.18]], 'conv2': [[0.0, 0.0, 0.0], [-0.67, 0.64, -0.43], [-1.01, 0.72, -1.11], [-1.18, 0.31, -0.25], [0.28, 0.52, 0.12], [-0.06, 1.24, -1.22], [-0.1, 0.69, -0.96]], 'norm': [[0.3333333134651184], [0.3333333134651184], [0.3333333134651184], [0.3333333134651184], [0.3333333134651184], [0.28867512941360474], [0.28867512941360474], [0.25], [0.25], [0.25], [0.28867512941360474], [0.28867512941360474], [0.3333333134651184], [0.28867512941360474], [0.28867512941360474], [0.3333333134651184], [0.3333333134651184], [0.28867512941360474], [0.28867512941360474], [0.25], [0.28867512941360474], [0.25], [0.25], [0.28867512941360474], [0.28867512941360474], [0.3333333134651184], [0.3333333134651184], [0.28867512941360474], [0.28867512941360474], [0.28867512941360474], [0.25], [0.28867512941360474], [0.28867512941360474], [0.25], [0.3535533845424652], [0.3535533845424652

In [29]:
out = torch.zeros([17,8],dtype=None)
# print(out)
edge_mul = torch.zeros([55,8],dtype=torch.int64)
# print(edge_mul)

i = 0
for x in edge_index[0]:
    for j in range(8):
        edge_mul[i,j] = x
    i = i+1
print(edge_mul.dtype)

print(torch.round(NM*100)/100)
out = out.scatter_add_(0, edge_mul, NM)
print(torch.round(out*100)/100)

i = 0
for row in out:
    out[i][0] = i
    i = i + 1

out_list = out.tolist()
print(out_list)



torch.int64
tensor([[ 0.0000, -0.0100,  0.1900,  0.1400,  0.3600,  0.1700,  0.1800,  0.3200],
        [ 0.0000, -0.0100,  0.1900,  0.1400,  0.3600,  0.1700,  0.1800,  0.3200],
        [ 0.3300, -0.0100,  0.1900,  0.1400,  0.3600,  0.1700,  0.1800,  0.3200],
        [ 0.3300, -0.0100,  0.1900,  0.1400,  0.3600,  0.1700,  0.1800,  0.3200],
        [ 0.6700, -0.0100,  0.1900,  0.1400,  0.3600,  0.1700,  0.1800,  0.3200],
        [ 0.5800, -0.0100,  0.1600,  0.1200,  0.3100,  0.1500,  0.1500,  0.2700],
        [ 0.8700,  0.0000,  0.1300,  0.2200, -0.0100, -0.2300,  0.0300,  0.0500],
        [ 0.7500,  0.0000,  0.1100,  0.1900, -0.0100, -0.2000,  0.0200,  0.0400],
        [ 0.7500,  0.0000,  0.1100,  0.1900, -0.0100, -0.2000,  0.0200,  0.0400],
        [ 1.0000,  0.0000,  0.1100,  0.1900, -0.0100, -0.2000,  0.0200,  0.0400],
        [ 1.1500,  0.0000,  0.1300,  0.2200, -0.0100, -0.2300,  0.0300,  0.0500],
        [ 1.1500,  0.0000,  0.1300,  0.2200, -0.0100, -0.2300,  0.0300,  0.0500],
    

In [23]:
print(clusters)

{'nodes': [{'id': '0', 'group': 0, 'f1': '0.0', 'h21': '-4.3', 'f2': '1.0', 'h22': '4.96', 'f3': '0.0', 'h23': '-5.16', 'h11': '0.0', 'h12': '1.26', 'h13': '0.95', 'h14': '1.66', 'h15': '0.4', 'h16': '1.11', 'h17': '1.62', 'fw11': '-0.02', 'fw12': '0.56', 'fw13': '0.42', 'fw14': '1.08', 'fw15': '0.51', 'fw16': '0.54', 'fw17': '0.95', 'br1': '-0.05', 'br2': '1.26', 'br3': '0.95', 'br4': '1.66', 'br5': '0.4', 'br6': '1.11', 'br7': '1.62'}, {'id': '1', 'group': 0, 'f1': '0.0', 'h21': '-4.3', 'f2': '1.0', 'h22': '4.96', 'f3': '0.0', 'h23': '-5.16', 'h11': '0.0', 'h12': '1.26', 'h13': '0.95', 'h14': '1.66', 'h15': '0.4', 'h16': '1.11', 'h17': '1.62', 'fw11': '-0.02', 'fw12': '0.56', 'fw13': '0.42', 'fw14': '1.08', 'fw15': '0.51', 'fw16': '0.54', 'fw17': '0.95', 'br1': '-0.05', 'br2': '1.26', 'br3': '0.95', 'br4': '1.66', 'br5': '0.4', 'br6': '1.11', 'br7': '1.62'}, {'id': '2', 'group': 0, 'f1': '0.0', 'h21': '-3.93', 'f2': '1.0', 'h22': '4.44', 'f3': '0.0', 'h23': '-4.72', 'h11': '0.0', 'h1

In [24]:
# print(clusters)
# import json
# with open('../../../Workspace/SQ\'19/Research/Graph_Viz/D3Code/GraphNetwork/Updated/GCN_Mutag_WeightsNew.json', 'w') as outfile:  
#    json.dump(clusters, outfile)
# outfile.close()